In [1]:
%reload_ext sql
import os

In [2]:
import sqlite3

In [3]:
survey_db_url = 'sqlite:////Users/reethsharma/git/cca/cs513_project/farmers_market.db'
%sql $survey_db_url

In [ ]:
%%sql
SELECT * FROM farmers_market limit 10;

### Next instructions are used to valdiate schema

In [7]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("test") \
      .getOrCreate()

22/06/29 19:57:20 WARN Utils: Your hostname, Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.1.78 instead (on interface en1)
22/06/29 19:57:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/06/29 19:57:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [22]:
df=spark.read.format("csv").option("header","true").load("dataset/farmers_market.csv",inferSchema = True)

In [20]:
df.createTempView("test")

AnalysisException: Temporary view 'test' already exists

In [11]:
spark.sql("select count(*) from test").show()

+--------+
|count(1)|
+--------+
|    8665|
+--------+



In [13]:
spark.sql("select * from test").head()

Row(FMID='1012063', MarketName=' Caledonia Farmers Market Association - Danville', Website='https://sites.google.com/site/caledoniafarmersmarket/', Facebook='https://www.facebook.com/Danville.VT.Farmers.Market/', Twitter=None, Youtube=None, OtherMedia=None, street=None, city='Danville ', County='Caledonia', State='Vermont', zip='05828', Season1Date='06/08/2016 to 10/12/2016', Season1Time='Wed: 9:00 AM-1:00 PM;', Season2Date=None, Season2Time=None, Season3Date=None, Season3Time=None, Season4Date=None, Season4Time=None, x='-72.140305', y='44.411013', Location=None, Credit='Y', WIC='Y', WICcash='N', SFMNP='Y', SNAP='N', Organic='Y', Bakedgoods='Y', Cheese='Y', Crafts='Y', Flowers='Y', Eggs='Y', Seafood='N', Herbs='Y', Vegetables='Y', Honey='Y', Jams='Y', Maple='Y', Meat='Y', Nursery='N', Nuts='N', Plants='Y', Poultry='Y', Prepared='Y', Soap='Y', Trees='Y', Wine='N', Coffee='Y', Beans='Y', Fruits='Y', Grains='N', Juices='Y', Mushrooms='N', PetFood='Y', Tofu='N', WildHarvested='N', updateTi

In [23]:
df.printSchema()

root
 |-- FMID: integer (nullable = true)
 |-- MarketName: string (nullable = true)
 |-- Website: string (nullable = true)
 |-- Facebook: string (nullable = true)
 |-- Twitter: string (nullable = true)
 |-- Youtube: string (nullable = true)
 |-- OtherMedia: string (nullable = true)
 |-- street: string (nullable = true)
 |-- city: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- zip: string (nullable = true)
 |-- Season1Date: string (nullable = true)
 |-- Season1Time: string (nullable = true)
 |-- Season2Date: string (nullable = true)
 |-- Season2Time: string (nullable = true)
 |-- Season3Date: string (nullable = true)
 |-- Season3Time: string (nullable = true)
 |-- Season4Date: string (nullable = true)
 |-- Season4Time: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- Credit: integer (nullable = true)
 |-- WIC: integer (nullable = true)
 |-- WIC

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset/main_farmers_market.csv')
df.dtypes

FMID               int64
MarketName        object
Website           object
Facebook          object
Twitter           object
Youtube           object
OtherMedia        object
street            object
city              object
County            object
State             object
zip               object
Season1Date       object
Season1Time       object
Season2Date       object
Season2Time       object
Season3Date       object
Season3Time       object
Season4Date       object
Season4Time       object
LAT              float64
LON              float64
Location          object
Credit             int64
WIC                int64
WICcash            int64
SFMNP              int64
SNAP               int64
Organic          float64
Bakedgoods       float64
Cheese           float64
Crafts           float64
Flowers          float64
Eggs             float64
Seafood          float64
Herbs            float64
Vegetables       float64
Honey            float64
Jams             float64
Maple            float64


Data Analysis and cleaning using Pandas

In [3]:
""" Date cleanup
1990
1990/01/01
1990/mar/01
Emoty to NaN
"""
df = df.apply(lambda col: pd.to_datetime(col, errors='ignore')
              if col.dtypes == object
              else col,
              axis=0)

df.dtypes

FMID                      int64
MarketName               object
Website                  object
Facebook                 object
Twitter                  object
Youtube                  object
OtherMedia               object
street                   object
city                     object
County                   object
State                    object
zip                      object
Season1Date              object
Season1Time              object
Season2Date              object
Season2Time              object
Season3Date              object
Season3Time              object
Season4Date              object
Season4Time              object
LAT                     float64
LON                     float64
Location                 object
Credit                    int64
WIC                       int64
WICcash                   int64
SFMNP                     int64
SNAP                      int64
Organic                 float64
Bakedgoods              float64
Cheese                  float64
Crafts  

In [8]:
df.head()

,FMID,MarketName,Website,Facebook,Twitter,Youtube,OtherMedia,street,city,County,...,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested,updateTime
0,1000001,Shelby Food and Craft Market,NaN,NaN,NaN,NaN,NaN,City Hall park,Shelby,Toole,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-01-01 00:00:00
1,1000003,Edmore Farmers Market,http://www.Edmore.org,https://www.facebook.com/EdmoreFarmersMarket,https://Twitter.com/edmoremarket,NaN,NaN,Corner of Lewis Street and Pine Street,Edmore,Montcalm,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2014-03-14 11:06:18
2,1000006,Westwood Village Farmers Market,http://www.westwoodonline.com,NaN,NaN,NaN,NaN,Weyburn Ave. & Westwood Blvd.,Los Angeles,Los Angeles,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-01-01 00:00:00
3,1000007,Santa Barbara La Cumbre Farmers Market,http://www.sbfarmersmarket.org,NaN,NaN,NaN,NaN,110 South Hope Avenue,Santa Barbara,Santa Barbara,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-01-01 00:00:00
4,1000008,Montrose Farmers Market,http://www.montrosefarmersmarket.com,https://www.facebook.com/MontroseFarmersMarket,NaN,NaN,NaN,Main St. and Uncompahgre Ave.,Montrose,Montrose,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2016-06-27 13:22:54


In [5]:
#Change column datatypes to boolean
df[df.columns[23:-1]]=df[df.columns[23:-1]].astype("boolean")

In [6]:
# Check number of columns have missing values
df.isna().sum()

FMID                0
MarketName          0
Website          3458
Facebook         4869
Twitter          7668
Youtube          8504
OtherMedia       8027
street            285
city               40
County            538
State               0
zip               944
Season1Date      3279
Season1Time      3140
Season2Date      8236
Season2Time      8251
Season3Date      8586
Season3Time      8590
Season4Date      8658
Season4Time      8658
LAT                29
LON                29
Location         5729
Credit              0
WIC                 0
WICcash             0
SFMNP               0
SNAP                0
Organic          5070
Bakedgoods       3023
Cheese           3023
Crafts           3023
Flowers          3023
Eggs             3023
Seafood          3023
Herbs            3023
Vegetables       3023
Honey            3023
Jams             3023
Maple            3023
Meat             3023
Nursery          3023
Nuts             3023
Plants           3023
Poultry          3023
Prepared  

In [12]:
df[df.columns[23:-1]]

,Credit,WIC,WICcash,SFMNP,SNAP,Organic,Bakedgoods,Cheese,Crafts,Flowers,...,Wine,Coffee,Beans,Fruits,Grains,Juices,Mushrooms,PetFood,Tofu,WildHarvested
0,False,False,False,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,True,True,False,True,True,True,True,False,True,True,...,False,False,False,True,False,False,False,False,False,False
2,False,False,False,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,False,False,False,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,True,False,False,False,True,True,True,True,True,True,...,False,True,True,True,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8660,True,False,False,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8661,True,False,False,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8662,True,False,False,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8663,True,False,False,False,False,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
df[df.columns[23:-1]]

In [7]:
df=df[df['LAT'].notna()]
df=df[df['LON'].notna()]

In [14]:
df['FMID'].value_counts(ascending=False)

1000001    1
1008437    1
1008460    1
1008457    1
1008456    1
          ..
1004111    1
1004109    1
1004108    1
1004107    1
1016770    1
Name: FMID, Length: 8636, dtype: int64

In [ ]:
import folium
# Create a map of U.S. farmers markets using latitude and longitude values
map_markets = folium.Map(location=[39.8283, -98.5795], zoom_start=5, popup='Portland, OR')

# add markers to map
for  lng,lat, market, state in zip(df['LAT'],df['LON'], df['MarketName'], df['State']):
    label = '{}, {}'.format(market, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#31cc34',
        fill_opacity=0.5,
        parse_html=False).add_to(map_markets)

map_markets

In [8]:
import sqlite3

conn = sqlite3.connect("farmers_market.db")

In [13]:
df.to_sql("farmers_market", conn, if_exists="replace")

8636

In [12]:
conn.execute(
    """
    create table farmers_market as
    select * from tmp
    """)

OperationalError: table farmers_market already exists